# PowerShell Data Recovery Script for SharePoint Online

## Overview
This PowerShell NoteBook serves as a practical guide to demonstrate the data recovery process from a backup provider to an operational WP365 system. 

These code sections accompany the document [WorkPoint Backup & Recovery - Technical Procedures](https://support.workpoint.dk/hc/en-us/articles/11427496217746-Backup-and-recovery-for-WorkPoint-365) by providing step-by-step walkthrough of code operations that require authentication, inspection and manipulation of WorkPoint365 data. 

## Important Notes
- Not all steps in the workflow require code. Refer to the document first and revert to the code as required.
- Several utility functions are defined and explained within the script and its comments to enhance understanding.
- This script is designed as an educational and reference resource and is not intended to be executed end to end without customization. 
- The system administrator is expected to adjust runtime parameters, including authentication, data sources, and recovery targets, to match the specific implementation.

## Script Usage
- Study each section of the script to understand the recovery process thoroughly.
- Adapt the script by interpreting and modifying the provided utility functions, variables, and configuration settings according to your organization's WorkPoint365 setup and backup provider.
- Execute specific portions of the script as needed for your data recovery scenarios.

## Disclaimer
This script is intended for educational and informational purposes. While it outlines the recovery process, it should be customized and tested in a controlled environment before use in a production setting. Careful consideration of authentication, security, and data integrity is paramount during the recovery process.

**Copyright (C) 2024 WorkPoint A/S. All rights reserved.**

In [16]:
# Retrieve and validate input parameters

## WorkPoint Solution Root: adjust to the URL of your WP365 Solution.
$WPSolutionUrl = $env:WP_SOLUTION_URL

## WorkPoint API endpoint: adjust for either ``api`` or ``api-test`` according to the implementation ring of your WP365 solution.
$WPApiUrl = $env:WP_API_URL

##  Some recovery operations interact w. WP365 API via authenticated REST requests.
# - These Application Registration MUST be adjusted for your Azure tenant and refer to an
#   enterprise application registration with ApplicationAccess to the Scope: WorkPoin365 WebApi (Test).
# - code requiring an AccessToken below can be adjusted if using an alternative Authentication provider. 
$AppReg = @{
    TenantId = $env:WP_APPREG_TENANT_ID
    ClientId = $env:WP_APPREG_CLIENT_ID
    ClientSecret = $env:WP_APPREG_CLIENT_SECRET
}

# validate input parameters
$Errors = @()
If (-not [System.Uri]::IsWellFormedUriString($WPSolutionUrl, 'Absolute')) {
    $Errors += "`$env:WP_SOLUTION_URL: contains invalid/missing Uri ($WPSolutionUrl)"
}
If (-not [System.Uri]::IsWellFormedUriString($WPApiUrl, 'Absolute')) {
    $Errors += "`$env:WP_API_URL: contains invalid/missing Uri ($WPApiUrl)"
}
#verify AppReg
If ((-not [guid]::TryParse($AppReg.TenantId, $([ref][guid]::Empty))) `
    -or (-not [guid]::TryParse($AppReg.ClientId, $([ref][guid]::Empty)))) {
        $Errors += "`$env:WP_APPREG_TENANT_ID / `$env:WP_APPREG_CLIENT_ID : contains invalid/missing GUID values ($AppReg)"
}
# abort if errors found.
If ($Errors.Length -gt 0) {
    Write-Error "Invalid input parameters found: verify: ($Errors -join ',')"
    Exit 1
}
Write-Host "Configuration loaded"

Configuration loaded


In [ ]:
# obtain an initial connection to the SP/WP Solution Url.
Connect-PnPOnline -Url $WPSolutionUrl -Interactive
$SharePointUrl = $WPSolutionUrl -split "/sites" | Select-Object -First 1
Write-Host "Connected to WP365: $($WPSolutionUrl) / $SharePointUrl"


### PowerShell Backup Inspection and Restoration Functions

#### Overview
This collection of PowerShell functions is designed to streamline the process of inspecting and restoring WP365 data from a backup provider. These functions provide a versatile toolkit for administrators and backup specialists to inspect and, if necessary, correct data recovered from a 3rd party BaaS provider.

**Disclaimer**: These functions are provided only as examples to demonstrate the types of manipulation necessary to recover and re-insert BaaS recovered data into a WP365 environment. Administrators are responsible for validating the functions are suitable for their WP365 environment and for the proper handling of sensitive data and compliance with data protection regulations.

In [8]:

<#
.SYNOPSIS
Converts a simple string into a GUID format by inserting delimiters.
Used to convert the non-delimited GUID format used in wpItemLocation into an actual GUID.

.PARAMETER InputString
The simple string that you want to convert into a GUID format.

.EXAMPLE
ConvertTo-Guid -InputString "c74c5e27123740fc82479b65b4033f69 "
# Output: "c74c5e27-1237-40fc-8247-9b65b4033f69"

#>
Function ConvertTo-Guid() {
    param (
        [Parameter(Position = 0)]
        [Object] $InputString
    )
    return ($InputString.Insert(8, "-").Insert(13, "-").Insert(18, "-").Insert(23, "-"))
}

Function ConvertFrom-Guid() {
    param (
        [Parameter(Position = 0)]
        [Object] $InputString
    )
    return $InputString -replace '-',''
}

<#
.SYNOPSIS
Inspects a BusinessModule SPList and identifies the properties used to manage an Entity in a Business Module hierarchy.

.PARAMETER BusinessModuleName
The name of the Business Module in string format e.g. "Locations"

.PARAMETER BusinessModuleId
The GUID of the BusinessModule in string format e.g. "c74c5e27-1237-40fc-8247-9b65b4033f69". 
This is used to compare the wpItemLocation value on the entities with the data recovered from backup. 

.EXAMPLE
Get-Entities -BusinessModuleName "Locations" -BusinessModuleId "c74c5e27-1237-40fc-8247-9b65b4033f69"
- OUTPUT: Title, fromBmId, fromEntityId, toBmId, toEntityId 
#>
Function Get-Entities() {
    param (
        [Parameter(Position = 0)]
        [Object] $BusinessModuleName,

        [Parameter(Position = 1)]
        [Object] $BusinessModuleId
    )
    
  
    # Use a Regular Expression to extract the BusinessModule and EntityId from a restored wpItemLocation path.
    $RgxLocation = '(?:([a-f,\d]{32});(\d+);)?([a-f,\d]{32});(\d+);$'


    # Get all Entity Items in the list (except MASTER)
    $EntityItems = @()
    $ListItems = Get-PnPListItem -List $BusinessModuleName -Query "<View><Query><Where><Gt><FieldRef Name='ID' /><Value Type='Number'>1</Value></Gt></Where></Query></View>"
    $toBmId = ConvertFrom-Guid -InputString $BusinessModuleId

    # Iterate, and identify inconsistent wpItemLocation values
    foreach ($Item in $ListItems)
    {
        # calculate the correct termination of the wpItemLocation field.
        $ItemLocation = "$($toBmId);$($Item.FieldValues.ID);"

        # found an item with an inconsistent wpItemLocation and Entity data, extract old BM/ItemId
        $M = $Item.FieldValues.wpItemLocation | Select-String -Pattern $RgxLocation -AllMatches

        $EntityItem = [PSCustomObject]@{
            Title = $Item.FieldValues.Title
            parentBmId = $M.Matches[0].Groups[1].Value
            parentEntityId = $M.Matches[0].Groups[2].Value
            fromBmId = $M.Matches[0].Groups[3].Value
            fromEntityId = $M.Matches[0].Groups[4].Value
            toBmId =  $toBmId 
            toEntityId  = $Item.FieldValues.ID
            wpSite  = $Item.FieldValues.wpSite 
            wpParent  = $Item.FieldValues.wpParent
            wpItemLocation  = $Item.FieldValues.wpItemLocation
            wpItemLocationValid = "$($Item.FieldValues.wpItemLocation)".EndsWith($ItemLocation)
        }

        $EntityItems += $EntityItem
    }

    return $EntityItems

}


<#
.SYNOPSIS
Used after a recovery operation to correct a Child Entity's wpParent value by using the wpItemLocation in the supplied ParentEntityList to find a
previous (now obsolete) ID
#>
Function Set-EntityParent {
    param (
        [Parameter(Position = 0)]
        [Object] $BusinessModuleName,
        
        [Parameter(Position = 1)]
        [Object] $EntityInfo,

        [Parameter(Position = 2)]
        [Object] $ParentEntityList,

        [Parameter(Position = 3)]
        [switch]$Force = $false
    )

    # Retrieve the list item from the Business Module, using the new EntityId
    $Item = Get-PnPListItem -List $BusinessModuleName -Id $EntityInfo.toEntityId -Fields "wpParent"

    #Check that the Entity has a new parentId, list item does NOT have a parent and that the values are inconsistent.
    If (($Force -eq $True) -or (($null -ne $EntityInfo.parentEntityId) -and ($null -eq $Item.FieldValues.wpParent) -and ($EntityInfo.parentEntityId -ne $Item.FieldValues.wpParent))) {

        #filter the ParentEntityList map, find the original parent Id to its new equivalent.
        $wpParent = ($ParentEntityList | Where-Object { $_.fromEntityId -eq $EntityInfo.parentEntityId }).toEntityId

       
        #If the values are different, update.
        If ($null -ne $wpParent) {
            Write-Host "Updating Parent on Entity ID:$($EntityInfo.toEntityId):'$($EntityInfo.Title)' - From:[$($Item.FieldValues.wpParent)] To:[$($wpParent)]" 
            Set-PnPListItem -List $BusinessModuleName -Identity $EntityInfo.toEntityId -Values @{"wpParent" = $wpParent} 
        } else {
            Write-Host "No Parent found Entity ID:$($EntityInfo.toEntityId):'$($EntityInfo.Title)' - skipping" 
        }

    } 

}

In [ ]:
<#
.SYNOPSIS
    Queries the WorkPoint Relations table to find any Relation Items that match the supplied Entity.

#>
Function Get-EntityRelations {
    param (
        [Parameter(Position = 1)]
        [Object] $EntityInfo,

        [Parameter(Position = 2)]
        [ValidateSet("From", "To")]
        [string]$UseId = "From"
    )

    $BmId = If ($UseId -eq "From") { $EntityInfo.fromBmId } Else { $EntityInfo.toBMId }
    $EntityId = If ($UseId -eq "From") { $EntityInfo.fromEntityId } Else { $EntityInfo.toEntityId }

    Write-Host "Finding Relations for BMId:$BmId Entity Id:$EntityId - (using '$UseId' values)"

    $Query = "<View>
    <Query>
        <Where> `
            <Or> `
                <And> `
                    <Eq><FieldRef Name=`"wpRelationAListId`" /><Value Type=`"Text`">$(ConvertTo-Guid($BmId))</Value></Eq>  `
                    <Eq><FieldRef Name=`"wpRelationAItemId`" /><Value Type=`"Number`">$($EntityId)</Value></Eq> `
                </And> `
                <And> `
                    <Eq><FieldRef Name=`"wpRelationBListId`" /><Value Type=`"Text`">$(ConvertTo-Guid($BmId))</Value></Eq>  `
                    <Eq><FieldRef Name=`"wpRelationBItemId`" /><Value Type=`"Number`">$($EntityId)</Value></Eq> `
                </And> `
            </Or> `
        </Where>
    </Query>
        <ViewFields> `
            <FieldRef Name=`"ID`" /> `
            <FieldRef Name=`"wpRelationAListId`" /> `
            <FieldRef Name=`"wpRelationAItemId`" /> `
            <FieldRef Name=`"wpRelationBListId`" /> `
            <FieldRef Name=`"wpRelationBItemId`" /> `
        </ViewFields> `
    </View>"

    $Relations = @()

    
    $Items = Get-PnPListItem -List "Relations" -Query $Query

    foreach ($Item in $Items) {
        $Relations += [PSCustomObject]@{
            ID = $($item.FieldValues.ID)
            wpRelationAListId = $($item.FieldValues.wpRelationAListId)
            wpRelationAItemId = $($item.FieldValues.wpRelationAItemId)
            wpRelationBListId = $($item.FieldValues.wpRelationBListId)
            wpRelationBItemId = $($item.FieldValues.wpRelationBItemId)
        }
    }

    Write-Host "- Found $($Relations.Count) relation(s)"
   
    return $Relations

}


In [ ]:
<#
.SYNOPSIS
Generates and executes a request to the WP365 ConnectToSite API for establishing a connection
between the Business Module Entity Item and the SPO site.

#>
Function Connect-EntityToSite {
    param (
        [string] $WorkPoint365Api = "https://api-test.workpoint365.com",
        
        [Parameter(Mandatory=$true)]
        [string] $SolutionUrl,

        [Parameter(Mandatory=$true)]
        [string] $AccessToken,
        
        [Parameter(Mandatory=$true)]
        [string] $BusinessModuleId,

        [Parameter(Mandatory=$true)]
        [int] $EntityId,

        [Parameter(Mandatory=$true)]
        [string] $SharePointUrl,
        
        [Parameter(Mandatory=$true)]
        [string] $SiteUrl,

        [Parameter(Mandatory=$false)]
        [string] $UpdateRelations = $false
    )

    If (-not ($SiteUrl -match "\/sites\/[^\/]+")) {
        Write-Host "Entity $EntityId has No Site To Connect $($SiteUrl)"
        return
    }
    
    #construct a URI to the ConnectToSite API
    $API =  "$($WorkPoint365Api)/api/BusinessModules/$($BusinessModuleId)/entities/$($EntityId)/ConnectToSite"
    
    $ReqData = @{
        SiteUrl = "$($SharePointUrl)$($SiteUrl)"
        SyncMasterSite = $False
        UpdateRelations = $UpdateRelations
    }

   return Invoke-WPAPI -API $API -SolutionUrl $SolutionUrl -AccessToken $AccessToken -ReqData $ReqData
}

<#
.SYNOPSIS
Generates and executes a request to the WP365 ReplaceRelationsByEntity API for 
replacing legacy relations for the specified entity.
#>
Function MapRelations {
    param (
        [string] $WorkPoint365Api = "https://api-test.workpoint365.com",
        [string] $SolutionUrl,
        [string] $AccessToken,
        [string] $fromBmId,
        [int] $fromEntityId,
        [string] $toBmId,
        [int] $toEntityId
    )
    
    #construct a URI to the ReplaceRelationsByEntity API
    $API =  "$($WorkPoint365Api)/api/Relations/ReplaceRelationsByEntity" 

    # Construct a Request to Map Legacy Relations (per API spec) to the new Entity Id
    $ReqData = @{
        fromBusinessModuleId = ConvertTo-Guid($fromBmId)
        fromEntityId = $fromEntityId
        toBusinessModuleId = ConvertTo-Guid($toBmId)
        toEntityId = $toEntityId
    }

    return Invoke-WPAPI -API $API -SolutionUrl $SolutionUrl -AccessToken $AccessToken -ReqData $ReqData
    
}

In [20]:
<#
.SYNOPSIS
Convenience function for retrieving an AccessToken from an Azure Application registration
using Tenant, ClientId and Secret

.PARAMETER AzureTenantId
The GUID TenantId obtained from http://portal.azure.com
e.g. "123e4567-e89b-12d3-a456-426655440000"

.PARAMETER AppRegClientId
The Application Registration Id obtained from the Azure Portal. Note the AppReg must have
ApplicationAccess to the "WorKPoint Web API" Enterprise Application in the Tenant
e.g. "a1b2c3d4-e5f6-4d3c-2b1a-1234567890ab"

.PARAMETER AppRegClientSecret
The client secret generated for the App Registration in the Azure Portal.
For further details see: https://learn.microsoft.com/en-us/entra/identity-platform/quickstart-register-app

.NOTES
Returns the AccessToken that can be used for subsequent WPAPI requests on the tenant.
#>
Function Get-WPAPIAccessToken() {
    param (
        [string] $AzureTenantId,
        [string] $AppRegClientId,
        [string] $AppRegClientSecret
    )

    #construct a Uri to the Azure Auth Service for the Tenant
    $ApiUri = "https://login.microsoftonline.com/$($AzureTenantId)/oauth2/v2.0/token"
    #construct the request payload including the AppReg credentials.
    $ReqData = @{
        grant_type = "client_credentials"
        client_id = $AppRegClientId
        client_secret =  $AppRegClientSecret
        scope = "https://workpoint365.com/wp365webapitest/.default"
    }

    #required headers for authentication scope w. WP API
    $Headers = @{
        scope = "https://workpoint365.com/wp365webapitest/user_impersonation"
    }
    
    #issue request to service & validate successful.
    $Response = Invoke-RestMethod -Uri $ApiUri -Method Post -Headers $Headers -Body $ReqData 
    If ($null -eq $Response.access_token) {
        throw "ERROR App Authentication Failed $($Response.Content)"
    }
    
    #construct a bearer token string and return for future requests.
    return $Response.access_token
}

<#
.SYNOPSIS
Convenience function for issuing an authenticated request to the WorkPoint API. 

.PARAMETER API
The full URL of the WorkPoint API to invoke, including the relevant API host
e.g. "https://api-test.workpoint365.com/api/Solution/Export/123"

.PARAMETER SolutionUrl
The full Solution URL of the WorkPoint solution
e.g. "https://m365x70229008.sharepoint.com/sites/WorkPoint"

.PARAMETER AccessToken
The bearer token retrieved from the Get-WPAPIAccessToken function. 
Must have ApplicationAccess to the WorkPoint API App Registration on the Tenant.

.PARAMETER ReqData
A Map containing the request data. Will be converted to JSON/body for POST requests.

.PARAMETER Method
HTTP Method verb to use for the request, Defaults to "POST"

.NOTES
Returns Response object from Invoke-RestMethod CmdLet
#>
Function Invoke-WPAPI {
    param (
        [Parameter(Position = 0, Mandatory=$true)]
        [string] $API,
        [Parameter(Position = 1, Mandatory=$true)]
        [string] $SolutionUrl,
        [Parameter(Position = 2, Mandatory=$true)]
        [string] $AccessToken,
        [Parameter(Position = 3, Mandatory=$false)]
        [object] $ReqData,
        [Parameter(Position = 4, Mandatory=$false)]
        [string] $Method = "POST"
    )

    
    $Headers = @{
        Accept = "application/json;odata=verbose"
        "Content-Type" = "application/json"
        WorkPoint365Url = $SolutionUrl
        Authorization = "Bearer $($AccessToken)"
    }
    
    $ReqData = $ReqData | ConvertTo-Json

    Write-Host "$($Method.ToString()):$API"
    Write-Host "- HEADERS:"
    $Headers | ConvertTo-Json | Write-Host
    Write-Host "- BODY:" 
    $ReqData | Write-Host

    

    #-Body $ReqData
    return Invoke-RestMethod -Uri $API -Method $Method -Headers $Headers -Body $ReqData
}


In [17]:
<#
.SYNOPSIS
Convenience function for decoding the output from the WP365 Export Function.

.PARAMETER EncodedString
The full ``Base64`` encoded string as retrieved from the API, https://api-test.workpoint365.com/api/Solution/Export/123

.PARAMETER OutFile
optional: The name of the file to create from the output (defaults to "Solution.zip")

#>
Function Save-WpExport() {
    [CmdletBinding()]
    param (
        [Parameter(Position = 0, ValueFromPipeline=$true)]
        [string]$EncodedString,
        [Parameter(Position = 1)]
        [string]$OutFile = "Solution.zip"
    )

    process {
  
        # Decode the BASE64 encoded export
        $decodedBytes = [System.Convert]::FromBase64String($EncodedString)
    
        # Save the decoded binary data to the file
        [IO.File]::WriteAllBytes($OutFile, $decodedBytes)

        return $OutFile

    }

}

## WorkPoint Recovery Steps
Code examples to accompany the workflow defined in the [Technical Procedures](https://support.workpoint.dk/hc/en-us/articles/11427496217746-Backup-and-recovery-for-WorkPoint-365)

Some notes:
- *Not all* procedure steps require code execution, refer to the document for the main flow and revert to the code elements if referenced in the document.
- Parameters for the Modules and connection may need to be changed according to the implementation.

### Step 7. Restore Entity Hierarchy
Here, we query a parent Business Module and identify any that have inconsistent ``wpItemLocation`` values.

In [9]:
# Identify the Parent Business Module by name, and obtain the list GUID.
$ParentBusinessModule = "Locations"
$ParentBusinessModuleId = (Get-PnPList -Identity $ParentBusinessModule).Id.ToString()

#call the custom Get-Entities function to retrieve all entities and compare current wpItemLocation with actual entity values.
$ParentEntities = Get-Entities -BusinessModuleName $ParentBusinessModule -BusinessModuleId $ParentBusinessModuleId
$NumInvalid = ($ParentEntities | Where-Object { $_.wpItemLocationValid -eq $false }).Count
Write-Host "$NumInvalid of $($ParentEntities.Count) $($ParentBusinessModule) Entities have inconsistent wpItemLocation"
$ParentEntities | Format-Table Title, fromBmId, fromEntityId, toBmId, toEntityId,  wpItemLocationValid

1 of 3 Locations Entities have inconsistent wpItemLocation

Title    fromBmId                         fromEntityId toBmId                           toEntityId wpItemLocationValid
-----    --------                         ------------ ------                           ---------- -------------------
Paris    315649f93dea4e5eae4265abb18e57b7 14           315649f93dea4e5eae4265abb18e57b7         14                True
London   315649f93dea4e5eae4265abb18e57b7 15           315649f93dea4e5eae4265abb18e57b7         15                True
New York 5afab17cb443411f8bd63a5722a37873 4            315649f93dea4e5eae4265abb18e57b7         16               False



### 7.a. Correct Parent Hierarchy
If a parent item has incorrect ``wpItemLocation``, this can be corrected by a simple save operation.
#### Strategy
- Observe the initial ``wpItemLocation`` values in the ``$ParentEntities`` collection (recovered from BaaS), _note_ these will be invalid.
- Invoke a simple save using the ``Set-PnPListItem`` (other _similar_ operation executed in _User Context_ will also work)
- Allow processing delay and fetch item again, note that WP365 services will have async repaired ``wpItemLocation``


In [12]:
# Pick and entity from the Parents list.
$Entity = $ParentEntities[2]
# Observe the BEFORE state of the Parent Entity object
(Get-PnPListItem -List $ParentBusinessModule -Id $Entity.toEntityId -Fields "ID", "Title","wpItemLocation").FieldValues


Key                                                                           Value
---                                                                           -----
ID                                                                               16
Title                                                                      New York
wpItemLocation 4f7ff1c5853b4956b4bc1e13ce1fa0cb;5afab17cb443411f8bd63a5722a37873;4;



In [ ]:
# Use ANY save operation (user context) to update the Entity
Set-PnpListItem -List $ParentBusinessModule -Identity $Entity.toEntityId -Values @{} -Force

In [ ]:
# Observe the CORRECTED wpItemLocation of the Parent Entity object
(Get-PnPListItem -List $ParentBusinessModule -Id $Entity.toEntityId -Fields "ID", "Title","wpItemLocation").FieldValues

#### 7.b. Correct Child Module Hierarchy
For child entities, the process is a little more complex. As the ``wpParent`` Lookup value is presumed lost in the recovery operation.
#### Strategy
- Parse the original ``wpItemLocation`` (recovered from BaaS)
- Identify the original wpParent (ID and bmID) using the components in ``wpItemLocation`` value.
- Lookup these values in the ``$ParentEntities`` collection.  
- Supply these values to the custom ``Set-EntityParent`` method to restore the parent lineage.
- Refresh the item and observe that the ``wpParent``  and ``wpItemLocation`` values are corrected by WP365 services.

In [14]:
# Identify the Child Business Module by name, and obtain the list GUID.
$ChildBusinessModule = "Projects"
$ChildBusinessModuleId = (Get-PnPList -Identity $ChildBusinessModule).Id.ToString()

# Call the custom Get-Entities function to retrieve all entities and compare current wpItemLocation with actual entity values.
$ChildEntities = Get-Entities -BusinessModuleName $ChildBusinessModule -BusinessModuleId $ChildBusinessModuleId
$NumInvalid = ($ChildEntities | Where-Object { $_.wpItemLocationValid -eq $false }).Count
Write-Host "$NumInvalid / $($ChildEntities.Count) of $ChildBusinessModule Entities have invalid wpItemLocation"
$ChildEntities | Format-Table Title, parentEntityId, toBmId, fromEntityId, toEntityId, wpItemLocationValid -AutoSize

1 / 6 of Projects Entities have invalid wpItemLocation

Title                  parentEntityId toBmId                           fromEntityId toEntityId wpItemLocationValid
-----                  -------------- ------                           ------------ ---------- -------------------
Baker Street           15             c74c5e27123740fc82479b65b4033f69 119                 119                True
Montparnasse           14             c74c5e27123740fc82479b65b4033f69 120                 120                True
Gare du Nord           14             c74c5e27123740fc82479b65b4033f69 121                 121                True
Charing Cross          15             c74c5e27123740fc82479b65b4033f69 122                 122                True
Grand Central Terminal 16             c74c5e27123740fc82479b65b4033f69 123                 123                True
Green Park             3              c74c5e27123740fc82479b65b4033f69 3                   124               False



In [15]:
#Selecting a Child Entity, we first observe its ``wpParent`` and ``wpItemLocation`` values.
$Entity = $ChildEntities[0]
#Observe the BEFORE state of an ChildEntity (Project)
(Get-PnPListItem -List $ChildBusinessModule -Id $Entity.toEntityId -Fields "ID", "Title","wpParent", "wpItemLocation").FieldValues


Key                                                                                                               Value
---                                                                                                               -----
ID                                                                                                                  119
Title                                                                                                      Baker Street
wpParent                                                                   Microsoft.SharePoint.Client.FieldLookupValue
wpItemLocation .2364295794aab8eade5f80c08a137;315649f93dea4e5eae4265abb18e57b7;15;c74c5e27123740fc82479b65b4033f69;119;



In [ ]:
# Use the ParentEntities map to check if wpParent is consistent with the mapping values, and if not - update 
Set-EntityParent -BusinessModuleName $ChildBusinessModule -Entity $Entity  -ParentEntityList $ParentEntities -Force

In [ ]:
#Observe the CORRECTED state of the same child entity, note wpParent and wpItemLocation values.
#!Note!, slight processing delay (~2-10s) as wpItemLocation is populated via WP365 Event Chain
(Get-PnPListItem -List $ChildBusinessModule -Id $ChildEntities[0].toEntityId -Fields "ID", "Title","wpParent", "wpItemLocation").FieldValues

### Step 8. Recover Entity Relationships
#### Strategy
- Obtain an ``access_token`` for the __App Registration__ for the tenant.
- _Observe_ the relations for an Entity using the ``Get-EntityRelations`` function.
- _Invoke_ the WP365 API ``ReplaceEntityRelations`` for the Entity, to update any existing references from ``old`` to ``new``.
- _Observe_ the old relations and confirm they now are linked to the updated entity item.

In [ ]:
# Authenticate w. Azure App Reg and obtain a Bearer Token that can be used for invoking WP365 API.
$AccessToken = Get-WPAPIAccessToken -AzureTenantId $AppReg.TenantId -AppRegClientId $AppReg.ClientId -AppRegClientSecret $AppReg.ClientSecret
Write-Host $AccessToken

In [ ]:
# Retrieve one of the original Entities (containing from/to BM/Entity ID values)
$Entity = $ParentEntities[1]
# Observe the ORIGINAL Relations that refer to the ORIGINAL fromBMId and entityID of a recovered Entity
Get-EntityRelations -EntityInfo $Entity -UseId "From" | Format-Table -AutoSize

In [ ]:
# Call the WP365 API /api/Relations/ReplaceRelationsByEntity - supplying the from/to ID values.
MapRelations -SolutionUrl $WPSolutionUrl -AccessToken $AccessToken -fromBmId $Entity.fromBmId `
             -fromEntityId $Entity.fromEntityId -toBmId $Entity.toBmId -toEntityId $Entity.toEntityId

In [ ]:
# Query the ORIGINAL Relations (AGAIN) to confirm that no Relations remain
Get-EntityRelations -EntityInfo $Entity -UseId "From" | Format-Table -AutoSize

In [ ]:
# Query the Relations - using the NEW BMId and Entity ID - Confirm relations mapped
Get-EntityRelations -EntityInfo $Entity -UseId "To" | Format-Table -AutoSize

### Step 10. Connect Entity Item and Site
#### Strategy
- Select a child entity that has been recovered from __BaaS_.
- _Observe_ the ``wp*`` columns on the the *Entity Item*, _Compare_ these to the ``PropertyBag`` entries on the Site itself.
- Invoke the ``ConnectToSite`` WP365 API to ensure the Entity Item and Site are correctly related.
- Compare Entity Item and Site again, to ensure they are consistent.

In [49]:
# Pick an entity from the List and observe its wp* column values.
$Entity = $ChildEntities[4]
$Entity


Title               : Grand Central Terminal
parentBmId          : 315649f93dea4e5eae4265abb18e57b7
parentEntityId      : 16
fromBmId            : c74c5e27123740fc82479b65b4033f69
fromEntityId        : 123
toBmId              : c74c5e27123740fc82479b65b4033f69
toEntityId          : 123
wpSite              : /sites/WorkPoint_Projects28
wpParent            : Microsoft.SharePoint.Client.FieldLookupValue
wpItemLocation      : 5162364295794aab8eade5f80c08a137;315649f93dea4e5eae4265abb18e57b7;16;c74c5e27123740fc82479b65b403
                      3f69;123;
wpItemLocationValid : True




In [50]:
# observe the CURRENT propertyBag entries of the connected site.
Write-Host "Properties of: $SharePointUrl$($Entity.wpSite)" 
$SiteConnection = Connect-PnPOnline -Url "$SharePointUrl$($Entity.wpSite)" -ReturnConnection -Interactive
Get-PnPPropertyBag -Connection $SiteConnection | Where-Object { $_.Key -Like "WP_*" } | Format-Table -AutoSize

Properties of: https://m365x70229008.sharepoint.com/sites/WorkPoint_Projects28

Key                           Value
---                           -----
WP_SITE_PARENT_LIST           c74c5e27-1237-40fc-8247-9b65b4033f69
WP_SITE_PARENT_SITECOLLECTION 18c2e0af-6bc1-4c38-8c61-65b4bc27c155
WP_SITE_PARENT_LIST_ITEM      123
WP_ITEM_LOCATION              5162364295794aab8eade5f80c08a137;315649f93dea4e5eae4265abb18e57b7;16;c74c5e27123740fc824.
WP_SITE_PARENT_WEB            94f30d4d-6efe-4ca8-bf2d-a384d7230cbc
WP_ROOT_SITECOLLECTION        https://m365x70229008.sharepoint.com/sites/workpoint



In [ ]:
# Call the Connect to Site API, 
Connect-EntityToSite -SolutionUrl $WPSolutionUrl -SharePointUrl $SharePointUrl -AccessToken $AccessToken `
                     -BusinessModuleId $Entity.toBmId -EntityId $Entity.toEntityId -SiteUrl $Entity.wpSite

In [71]:
# observe the UPDATED properties of the connected entity site.
Write-Host "Properties of: $SharePointUrl$($Entity.wpSite) - AFTER ConnectToSite" 
Get-PnPPropertyBag -Connection $SiteConnection | Where-Object { $_.Key -Like "WP_*" } | Format-Table -AutoSize

Properties of: https://m365x70229008.sharepoint.com/sites/WorkPoint_Projects28 - AFTER ConnectToSite

Key                                                                                                               Value
---                                                                                                               -----
WP_SITE_PARENT_LIST_ITEM                                                                                            123
WP_ROOT_SITECOLLECTION                                             https://m365x70229008.sharepoint.com/sites/workpoint
WP_SITE_PARENT_SITECOLLECTION                                                      18c2e0af-6bc1-4c38-8c61-65b4bc27c155
WP_SITE_PARENT_WEB                                                                 94f30d4d-6efe-4ca8-bf2d-a384d7230cbc
WP_SITE_PARENT_LIST                                                                c74c5e27-1237-40fc-8247-9b65b4033f69
WP_ITEM_LOCATION              .ade5f80c08a137;315649f93dea

### Step 12. Select Content Restore
##### Strategy:
- Pick an entity that has been recovered and confirm that the ``wpItemLocation`` value in the ``Entity Item`` Site and ``Entity Site`` are consistent.
- We execute a BAAS restore of the contents, writing back data from the BaaS provider to the ``Documents`` library.
- We run the ``EnsureAllItemLocations`` function in *WorkPoint Admin* to ensure all ``wpItemLocation`` values are consistent.
- _Observe_ the ``wpItemLocation`` values of the recovery.

In [29]:
 $Entity = $ChildEntities[4]
 $Entity


Title               : Grand Central Terminal
parentBmId          : 315649f93dea4e5eae4265abb18e57b7
parentEntityId      : 16
fromBmId            : c74c5e27123740fc82479b65b4033f69
fromEntityId        : 123
toBmId              : c74c5e27123740fc82479b65b4033f69
toEntityId          : 123
wpSite              : /sites/WorkPoint_Projects28
wpParent            : Microsoft.SharePoint.Client.FieldLookupValue
wpItemLocation      : 5162364295794aab8eade5f80c08a137;315649f93dea4e5eae4265abb18e57b7;16;c74c5e27123740fc82479b65b403
                      3f69;123;
wpItemLocationValid : True




In [30]:
# observe the CURRENT of an Entity Site site.
Write-Host "Properties of: $SharePointUrl$($Entity.wpSite)" 
$SiteConnection = Connect-PnPOnline -Url "$SharePointUrl$($Entity.wpSite)" -ReturnConnection -Interactive
$wpItemLocation = Get-PnPPropertyBag -Connection $SiteConnection -Key 'WP_ITEM_LOCATION'
# observe what the Entity site should be
$wpItemLocation

Properties of: https://m365x70229008.sharepoint.com/sites/WorkPoint_Projects28
5162364295794aab8eade5f80c08a137;315649f93dea4e5eae4265abb18e57b7;16;c74c5e27123740fc82479b65b4033f69;123;


In [61]:
# gather a collection of items from the "Documents" library in a site (that has had content restored from a BaaS operation).
Function Get-WPSiteItems() {

    param (
        [Parameter(Position = 0)]
        [object] $Connection,
        [Parameter(Position = 1, Mandatory=$false)]
        [string]$ListName = "Documents",
        [Parameter(Position = 2, Mandatory=$false)]
        [string]$Return = "Documents"
    )

    $wpItemLocation = Get-PnPPropertyBag -Connection $Connection -Key 'WP_ITEM_LOCATION'
    
    $Items = Get-PnPListItem -Connection $Connection -List $ListName | `
    Select-Object id,   @{label="Filename";expression={$_.FieldValues.FileLeafRef}}, `
    @{label="wpItemLocation";expression={$_.FieldValues.wpItemLocation}}, `
    @{label="wpItemLocationValid";expression={$_.FieldValues.wpItemLocation -eq $wpItemLocation}}
    
    $NumInvalid = ($Items | Where-Object { $_.wpItemLocationValid -eq $false }).Count
    Write-Host "Collected $($Items.Count) Documents from $($Connection.Url) - ($NumInvalid) have invalid wpItemLocation"
    
    return $Items
}

In [63]:
$Items = Get-WPSiteItems -Connection $SiteConnection
$Items | Format-Table -Property Id, Filename, wpItemLocationValid

Collected 26 Documents from https://m365x70229008.sharepoint.com/sites/workpoint_projects28 - (25) have invalid wpItemLocation

Id Filename                            wpItemLocationValid
-- --------                            -------------------
 6 Closure-003.docx                                   True
 7 System Architecture Diagram.docx                  False
 8 Server Configuration Checklist.docx               False
 9 Vendor Contact List.docx                          False
10 Change Request Form.docx                          False
11 IT Project Plan.docx                              False
12 User Training Manual.docx                         False
15 Project Timeline.docx                             False
16 Risk Assessment Report.docx                       False
17 System Maintenance Schedule.docx                  False
18 Meeting Minutes 2023-09-15.docx                   False
19 Quality Assurance Checklist.docx                  False
20 Technical Support Handbook.docx            

In [65]:
# Experiment: force update one item (in user context), observe the ``wpItemLocation`` value is self corrected.
# pick an item that has an invalid wpItemLocation
$Item = ($Items | Where-Object {$_.wpItemLocationValid -eq $false})[0]
$ItemId = $Item.Id
Set-PnpListItem -Connection $SiteConnection -List "Documents" -Identity $ItemId -Values @{wpItemLocation = $null} -Force


Id    Title                                              GUID
--    -----                                              ----
7                                                        b1a37b65-a940-4468-93cc-9092cbf86fef



In [70]:
# fetch the items again, and filter for the object we just updated, refresh periodically
# note that:  -  wpItemLocation is initially blank, but will be corrected (async) by wp events job
$Items = Get-WPSiteItems -Connection $SiteConnection
$Item = ($Items | Where-Object {$_.Id -eq $ItemId})[0]
$Item | Format-List


Collected 26 Documents from https://m365x70229008.sharepoint.com/sites/workpoint_projects28 - (25) have invalid wpItemLocation

Id                  : 7
Filename            : System Architecture Diagram.docx
wpItemLocation      : 
wpItemLocationValid : False




In [35]:
$Items | Format-Table -Property Id, Filename, wpItemLocationValid


Id Filename                            wpItemLocationValid
-- --------                            -------------------
 6 Closure-003.docx                                  False
 7 System Architecture Diagram.docx                  False
 8 Server Configuration Checklist.docx               False
 9 Vendor Contact List.docx                          False
10 Change Request Form.docx                          False
11 IT Project Plan.docx                              False
12 User Training Manual.docx                         False
15 Project Timeline.docx                             False
16 Risk Assessment Report.docx                       False
17 System Maintenance Schedule.docx                  False
18 Meeting Minutes 2023-09-15.docx                   False
19 Quality Assurance Checklist.docx                  False
20 Technical Support Handbook.docx                   False
21 Backup and Recovery Plan.docx                     False
22 Security Policy and Guidelines.docx               Fa

### Solution Export
#### Strategy
- Demonstrate use of the WP365 API ``/api/Solution/Export``.
- Show how the archive content of a previous Export Job can be downloaded, decoded and stored using the API svc.

In [ ]:
$SPHostUrl = "https://wp365test.sharepoint.com/sites/NN0001UK/"

In [ ]:
# Authenticate with the Azure App Registration
$AccessToken = Get-WPAPIAccessToken -AzureTenantId $AppReg.TenantId `
                                    -AppRegClientId $AppReg.ClientId `
                                    -AppRegClientSecret $AppReg.ClientSecret
$AccessToken

In [ ]:
# Generate an authenticated request to the WP365 API, referencing the TemplateId and Solution
$R = Invoke-WPApi -Api "https://api-test.workpoint365.com/api/Solution/Export/633" -SolutionUrl $SPHostUrl -AccessToken $AccessToken -Method "GET"
Write-Host "- Response:"
$R

In [ ]:
#Output the decoded response to Zip.
$R | Save-WpExport